# Lab | Advanced SQL

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///../20-lab-sql-select/publications.db

In [13]:
%%sql
select * from sqlite_master

 * sqlite:///../20-lab-sql-select/publications.db
Done.


type,name,tbl_name,rootpage,sql
table,authors,authors,2,"CREATE TABLE `authors` ( `au_id` varchar(11) NOT NULL, `au_lname` varchar(40) NOT NULL, `au_fname` varchar(20) NOT NULL, `phone` char(12) NOT NULL, `address` varchar(40) DEFAULT NULL, `city` varchar(20) DEFAULT NULL, `state` char(2) DEFAULT NULL, `zip` char(5) DEFAULT NULL, `contract` integer NOT NULL, PRIMARY KEY (`au_id`))"
index,sqlite_autoindex_authors_1,authors,3,None
table,discounts,discounts,4,"CREATE TABLE `discounts` ( `discounttype` varchar(40) NOT NULL, `stor_id` char(4) DEFAULT NULL, `lowqty` integer DEFAULT NULL, `highqty` integer DEFAULT NULL, `discount` decimal(4,2) NOT NULL, CONSTRAINT `discounts_ibfk_1` FOREIGN KEY (`stor_id`) REFERENCES `stores` (`stor_id`) ON DELETE CASCADE ON UPDATE CASCADE)"
table,employee,employee,5,"CREATE TABLE `employee` ( `emp_id` char(9) NOT NULL, `fname` varchar(20) NOT NULL, `minit` char(1) DEFAULT NULL, `lname` varchar(30) NOT NULL, `job_id` integer NOT NULL, `job_lvl` integer DEFAULT NULL, `pub_id` char(4) NOT NULL, `hire_date` datetime NOT NULL, PRIMARY KEY (`emp_id`), CONSTRAINT `employee_ibfk_1` FOREIGN KEY (`job_id`) REFERENCES `jobs` (`job_id`) ON DELETE CASCADE ON UPDATE CASCADE, CONSTRAINT `employee_ibfk_2` FOREIGN KEY (`pub_id`) REFERENCES `publishers` (`pub_id`) ON DELETE CASCADE ON UPDATE CASCADE)"
index,sqlite_autoindex_employee_1,employee,6,None
table,jobs,jobs,7,"CREATE TABLE `jobs` ( `job_id` integer NOT NULL, `job_desc` varchar(50) NOT NULL, `min_lvl` integer NOT NULL, `max_lvl` integer NOT NULL, PRIMARY KEY (`job_id`))"
table,pub_info,pub_info,8,"CREATE TABLE `pub_info` ( `pub_id` char(4) NOT NULL, `logo` longblob DEFAULT NULL, `pr_info` longtext DEFAULT NULL, PRIMARY KEY (`pub_id`), CONSTRAINT `pub_info_ibfk_1` FOREIGN KEY (`pub_id`) REFERENCES `publishers` (`pub_id`) ON DELETE CASCADE ON UPDATE CASCADE)"
index,sqlite_autoindex_pub_info_1,pub_info,9,None
table,publishers,publishers,34,"CREATE TABLE `publishers` ( `pub_id` char(4) NOT NULL, `pub_name` varchar(40) DEFAULT NULL, `city` varchar(20) DEFAULT NULL, `state` char(2) DEFAULT NULL, `country` varchar(30) DEFAULT NULL, PRIMARY KEY (`pub_id`))"
index,sqlite_autoindex_publishers_1,publishers,35,None


## Challenge 1 - Most Profiting Authors

In this challenge let's have a close look at the bonus challenge of the previous *SQL SELECT* lab -- **who are the top 3 most profiting authors**? Even if you have solved or think you have solved that problem in the previous lab, please still complete this challenge because the step-by-step guidances are helpful to train your problem-solving thinking.

In order to solve this problem, it is important for you to keep the following points in mind:

* In table `sales`, a title can appear several times. The royalties need to be calculated for each sale.

* Despite a title can have multiple `sales` records, the advance must be calculated only once for each title.

* In your eventual solution, you need to sum up the following profits for each individual author:
    * All advances which is calculated exactly once for each title.
    * All royalties in each sale.

Therefore, you will not be able to achieve the goal with a single SELECT query. Instead, you will need to follow several steps in order to achieve the eventual solution. Below is an overview of the steps:

1. Calculate the royalty of each sale for each author.

1. Using the output from Step 1 as a sub-table, aggregate the total royalties for each title for each author.

1. Using the output from Step 2 as a sub-table, calculate the total profits of each author by aggregating the advances and total royalties of each title.

Below we'll guide you through each step. In your `solutions.ipynb`, please include the SELECT queries of each step so that your TA can review your problem-solving process.

### Step 1: Calculate the royalties of each sales for each author

Write a SELECT query to obtain the following output:

* Title ID
* Author ID
* Royalty of each sale for each author
    * The formula is:
        ```
        sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100
        ```
    * Note that `titles.royalty` and `titleauthor.royaltyper` are divided by 100 respectively because they are percentage numbers instead of floats.

In the output of this step, each title may appear more than once for each author. This is because a title can have more than one sales.

sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100

In [48]:
%%sql
select titleauthor.title_id as TITLE_ID, titleauthor.au_id as AUTHOR_ID,
(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) as SALES_ROYALTY
from titleauthor
left join titles
    on titleauthor.title_id=titles.title_id
left join sales
    on titleauthor.title_id=sales.title_id




 * sqlite:///../20-lab-sql-select/publications.db
Done.


TITLE_ID,AUTHOR_ID,SALES_ROYALTY
PS3333,172-32-1176,29.984999999999996
BU1032,213-46-8915,3.9979999999999993
BU1032,213-46-8915,7.995999999999999
BU2075,213-46-8915,25.116000000000003
PC1035,238-95-7766,110.16
BU1111,267-41-2394,11.95
TC7777,267-41-2394,8.994
BU7832,274-80-9391,29.984999999999996
BU1032,409-56-7008,5.996999999999999
BU1032,409-56-7008,11.993999999999998


### Step 2: Aggregate the total royalties for each title for each author

Using the output from Step 1, write a query to obtain the following output:

* Title ID
* Author ID
* Aggregated royalties of each title for each author
    * Hint: use the *SUM* subquery and group by both `au_id` and `title_id`

In the output of this step, each title should appear only once for each author.

In [34]:
%%sql
with cte as
(select titleauthor.title_id as TITLE_ID, titleauthor.au_id as AUTHOR_ID,
(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) as SALES_ROYALTY
from titleauthor
left join titles
    on titleauthor.title_id=titles.title_id
left join sales
    on titleauthor.title_id=sales.title_id)
select TITLE_ID,  AUTHOR_ID, sum(SALES_ROYALTY) as SALES_ROYALTY
from cte
group by TITLE_ID,AUTHOR_ID
order by SALES_ROYALTY desc

 * sqlite:///../20-lab-sql-select/publications.db
Done.


TITLE_ID,AUTHOR_ID,SALES_ROYALTY
PC1035,238-95-7766,110.16
TC3218,807-91-6654,83.8
PS2091,899-46-2035,70.95599999999999
PS2091,998-72-3567,70.95599999999999
PC8888,427-17-2319,50
PC8888,846-92-7186,50
TC4203,648-92-1872,33.46
PS1372,756-30-7391,32.385
PS3333,172-32-1176,29.984999999999996
BU7832,274-80-9391,29.984999999999996


### Step 3: Calculate the total profits of each author

Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the output from Step 2, write a query to obtain the following output:

* Author ID
* Profits of each author by aggregating the advance and total royalties of each title

Sort the output based on a total profits from high to low, and limit the number of rows to 3.

In [49]:
%%sql
with subquery as(
with cte as
(select titleauthor.title_id as TITLE_ID, titleauthor.au_id as AUTHOR_ID,
(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) as SALES_ROYALTY
from titleauthor
left join titles
    on titleauthor.title_id=titles.title_id
left join sales
    on titleauthor.title_id=sales.title_id)
select TITLE_ID,  AUTHOR_ID, sum(SALES_ROYALTY) as SALES_ROYALTY
from cte
group by TITLE_ID,AUTHOR_ID)
select AUTHOR_ID, SALES_ROYALTY+titles.advance as PROFITS
from subquery
inner join titles
     on subquery.title_id=Titles.title_id
order by profits desc
limit 3

 * sqlite:///../20-lab-sql-select/publications.db
Done.


AUTHOR_ID,PROFITS
722-51-5454,15021.528
899-46-2035,15007.176
213-46-8915,10150.116


## Challenge 2

Elevating from your solution in Challenge 1 , create a table named `most_profiting_authors` to hold the data about the most profiting authors. The table should have 2 columns:

* `au_id` - Author ID
* `profits` - The profits of the author aggregating the advances and royalties

Include your solution in `solutions.ipynb`.

In [50]:
%%sql
create table "most_profiting_authors" as 
with subquery as(
with cte as
(select titleauthor.title_id as TITLE_ID, titleauthor.au_id as AUTHOR_ID,
(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) as SALES_ROYALTY
from titleauthor
left join titles
    on titleauthor.title_id=titles.title_id
left join sales
    on titleauthor.title_id=sales.title_id)
select TITLE_ID,  AUTHOR_ID, sum(SALES_ROYALTY) as SALES_ROYALTY
from cte
group by TITLE_ID,AUTHOR_ID)
select AUTHOR_ID, SALES_ROYALTY+titles.advance as PROFITS
from subquery
inner join titles
     on subquery.title_id=Titles.title_id
order by profits desc


 * sqlite:///../20-lab-sql-select/publications.db
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [52]:
%%sql
select * from most_profiting_authors

 * sqlite:///../20-lab-sql-select/publications.db
Done.


AUTHOR_ID,PROFITS
722-51-5454,15021.528
899-46-2035,15007.176
213-46-8915,10150.116
427-17-2319,8050
846-92-7186,8050
672-71-3249,8011.992
267-41-2394,8008.994
472-27-2349,8008.994
238-95-7766,7110.16
807-91-6654,7083.8
